In [1]:
from os import path
import MDAnalysis as mda
from bentdna.PDB import PDBReader, PDBWriter
from bentdna.find_haxis_curve import FindHelixAgent, PrepareHelix

### Part 0: Prepare Required file

In [2]:
host = 'pnas_16mer'
n_bp = 16
find_helix_folder = '/home/yizaochen/codes/bentdna/tests'

prep_helix = PrepareHelix(find_helix_folder, host, n_bp)
print(f'cd {prep_helix.workfolder}')

/home/yizaochen/codes/bentdna/tests/pnas_16mer/input exists
/home/yizaochen/codes/bentdna/tests/pnas_16mer/output exists
cd /home/yizaochen/codes/bentdna/tests/pnas_16mer


In [3]:
prep_helix.copy_input_xtc()
prep_helix.copy_input_pdb()

cp /home/yizaochen/codes/dna_rna/all_systems/pnas_16mer/bdna+bdna/input/allatoms/bdna+bdna.all.xtc /home/yizaochen/codes/bentdna/tests/pnas_16mer/input/bdna+bdna.all.xtc
cp /home/yizaochen/codes/dna_rna/all_systems/pnas_16mer/bdna+bdna/input/allatoms/bdna+bdna.npt4.all.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/input/bdna+bdna.npt4.all.pdb


### Cut trajectory for testcase
`gmx trjcat -f bdna+bdna.all.xtc -o temp.xtc -e 1000`  
`mv temp.xtc bdna+bdna.all.xtc`

### Part 1: assign number of base-pairs

In [3]:
n_bp = 16
pdb_in = path.join(prep_helix.input_folder, 'bdna+bdna.npt4.all.pdb')
xtc_in = path.join(prep_helix.input_folder, 'bdna+bdna.all.xtc')

### Part 2: Convert xtc to dcd

In [4]:
cmd = f'vmd -pdb {prep_helix.pdb_in} {prep_helix.xtc_in}'
print(cmd)

# In vmd tkconsole
dcd_out = path.join(prep_helix.input_folder, 'bdna+bdna.0_1ns.10frames.dcd')
print('In vmd tkconsole')
cmd = f'animate write dcd {dcd_out} beg 1 end 11 waitfor all'
print(cmd)

vmd -pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/input/bdna+bdna.npt4.all.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/input/bdna+bdna.all.xtc
In vmd tkconsole
animate write dcd /home/yizaochen/codes/bentdna/tests/pnas_16mer/input/bdna+bdna.0_1ns.10frames.dcd beg 1 end 11 waitfor all


### Part 3: Change B-chain ID from 1-16 to 17-32

In [5]:
pdb_modified = path.join(prep_helix.input_folder, 'bdna_modi.pdb')
# check pdb, to see whether require to change resid
cmd = f'vim {prep_helix.pdb_in}'
print(cmd)

vim /home/yizaochen/codes/bentdna/tests/pnas_16mer/input/bdna+bdna.npt4.all.pdb


In [11]:
reader = PDBReader(pdb_in, segid_exist=True)
atgs = reader.get_atomgroup()

# Change resid
resid_offset = n_bp
for atom in atgs:
    if atom.segid == 'B':
        atom.resid += resid_offset
        

writer = PDBWriter(pdb_modified, atgs)
writer.write_pdb()

Write PDB: /home/yizaochen/codes/bentdna/tests/pnas_16mer/input/bdna_modi.pdb


### Part 4: Initialize FindHelixAgent

In [6]:
f_agent = FindHelixAgent(prep_helix.workfolder, pdb_modified, dcd_out, n_bp)

/home/yizaochen/codes/bentdna/tests/pnas_16mer/pdbs_allatoms exists
/home/yizaochen/codes/bentdna/tests/pnas_16mer/curve_workdir_0_11 exists
/home/yizaochen/codes/bentdna/tests/pnas_16mer/pdbs_haxis exists
mkdir /home/yizaochen/codes/bentdna/tests/pnas_16mer/haxis_smooth
There are 11 frames.


/home/yizaochen/miniconda3/envs/bentdna/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:330: UserWarning: Element information is absent or missing for a few atoms. Elements attributes will not be populated.
  warnings.warn("Element information is absent or missing for a few "


### Part 5: Extract single pdb from dcd

In [16]:
f_agent.extract_pdb_allatoms()

### Part 6: Execute Curve+ and Convert to H-axis pdb

In [7]:
f_agent.curveplus_find_smooth_haxis()
#f_agent.curveplus_find_haxis()

cp /home/yizaochen/codes/bentdna/tests/pnas_16mer/curve_workdir_0_11/r+bdna_X.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/haxis_smooth/haxis.smooth.0.pdb
cp /home/yizaochen/codes/bentdna/tests/pnas_16mer/curve_workdir_0_11/r+bdna_X.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/haxis_smooth/haxis.smooth.1.pdb
cp /home/yizaochen/codes/bentdna/tests/pnas_16mer/curve_workdir_0_11/r+bdna_X.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/haxis_smooth/haxis.smooth.2.pdb
cp /home/yizaochen/codes/bentdna/tests/pnas_16mer/curve_workdir_0_11/r+bdna_X.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/haxis_smooth/haxis.smooth.3.pdb
cp /home/yizaochen/codes/bentdna/tests/pnas_16mer/curve_workdir_0_11/r+bdna_X.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/haxis_smooth/haxis.smooth.4.pdb
cp /home/yizaochen/codes/bentdna/tests/pnas_16mer/curve_workdir_0_11/r+bdna_X.pdb /home/yizaochen/codes/bentdna/tests/pnas_16mer/haxis_smooth/haxis.smooth.5.pdb
cp /home/yizaochen/codes/bentdna/t

### Part 7: Use VMD to show

In [16]:
"""
cmd = 'vmd'
print(cmd)

haxis_tcl = '/home/yizaochen/codes/na_mechanics/make_haxis.tcl'
cmd = f'source {haxis_tcl}'
print(cmd)

start = 0
end = 10000
cmd = f'read_all_pdb_files {start} {end}'
print(cmd)

haxis_dcd = path.join(prep_helix.input_folder, 'haxis.dcd')
cmd = f'animate write dcd {haxis_dcd} beg {start} end {end} waitfor all'
print(cmd)
"""

vmd
source /home/yizaochen/codes/na_mechanics/make_haxis.tcl
read_all_pdb_files 0 10000
animate write dcd ../haxis.dcd beg 0 end 10000 waitfor all


In [17]:
"""
pdb_ref = path.join(prep_helix.workfolder, 'pdbs_haxis', 'haxis.0.pdb')
cmd = f'vmd -pdb {pdb_ref} {haxis_dcd}'
print(cmd)

cmd = f'mol new {pdb_modified}'
print(cmd)

cmd = f'mol addfile {dcd_out} 1'
print(cmd)
"""

vmd -pdb haxis.0.pdb ../haxis.dcd
mol new /home/yizaochen/codes/dna_rna/length_effect/ipynbs/find_helical_axis/pnas_amber/bdna_modi.pdb
mol addfile /home/yizaochen/codes/dna_rna/length_effect/ipynbs/find_helical_axis/pnas_amber/bdna+bdna.0_1000ns.10000frames.dcd 1


### Part 8: rm pdb_allatoms

In [4]:
"""
allpdbs = path.join(prep_helix.workfolder, 'pdbs_allatoms', '*')
cmd = f'rm {allpdbs}'
print(cmd)
"""

rm /home/yizaochen/codes/dna_rna/length_effect/find_helical_axis/pnas_16mer/pdbs_allatoms/*


### Useful commands

- cat 0.pdb | grep 'ATOM      1'